Created on Thu Apr 18 05:43:21 2024

@author: Santiago D'hers

Use:

- This script will create autolabels analyzing position files

Requirements:

- The position.csv files processed by 1-Manage_H5.py
- The desired model trained with 3a-Create_Models.py

In [52]:
import os
from glob import glob
import pandas as pd
import numpy as np

import tensorflow as tf

import joblib
from keras.models import load_model

import datetime

In [53]:
# Set the variables before starting
desktop = 'C:/Users/dhers/Desktop'
STORM_folder = os.path.join(desktop, 'STORM/models')

# State your path:
path = r'C:\Users\dhers\OneDrive - UBA\workshop'
experiment = r'TeNOR'

before, after = 3, 3
frames = before + after + 1

all_position = glob(os.path.join(path, experiment,"T*/position/*position.csv"))

today = datetime.datetime.now()
# use_model_date = today.date()
use_model_date = '2024-10-16'

objects = ['obj_1', 'obj_2']
bodyparts = ['nose', 'L_ear', 'R_ear', 'head', 'neck', 'body']
rescale = True
reshape = True # True for wide models

In [54]:
# Load the saved model from file

# loaded_model = load_model(os.path.join(STORM_folder, f'simple/model_simple_{use_model_date}.keras'))
loaded_model = load_model(os.path.join(STORM_folder, f'wide/model_wide_{use_model_date}.keras'))
# loaded_model = joblib.load(os.path.join(STORM_folder, f'RF/model_RF_{use_model_date}.pkl'))

In [55]:
def recenter(df, point, bodyparts):
    # Create a copy of the original dataframe
    df_copy = df.copy()
    bodypart_columns = []
    
    for bodypart in bodyparts:
        # Subtract point_x from columns ending in _x
        x_cols = [col for col in df_copy.columns if col.endswith(f'{bodypart}_x')]
        df_copy[x_cols] = df_copy[x_cols].apply(lambda col: col - df_copy[f'{point}_x'])
        
        # Subtract point_y from columns ending in _y
        y_cols = [col for col in df_copy.columns if col.endswith(f'{bodypart}_y')]
        df_copy[y_cols] = df_copy[y_cols].apply(lambda col: col - df_copy[f'{point}_y'])
        
        # Collect bodypart columns
        bodypart_columns.extend(x_cols)
        bodypart_columns.extend(y_cols)
        
    return df_copy[bodypart_columns]

In [56]:
def reshape(df, back = before, forward = after): # This function reshapes data for LSTM models
    
    reshaped_df = []
    
    for i in range(0, back):
        reshaped_df.append(df[: 1 + back + forward])
            
    for i in range(back, len(df) - forward):
        reshaped_df.append(df[i - back : 1 + i + forward])
    
    for i in range(len(df) - forward, len(df)):
        reshaped_df.append(df[-(1 + back + forward):])
    
    return reshaped_df

In [57]:
def use_model(position, model, objects = ['obj_1', 'obj_2'], bodyparts = ['nose', 'L_ear', 'R_ear', 'head', 'neck', 'body'], recentering = True, reshaping = False):
    
    if recentering:
        dfs = []
        for obj in objects:
            recentered = recenter(position, obj, bodyparts)
            dfs.append(recentered)
        position = pd.concat(dfs,ignore_index=True)
    
    if reshaping:
        position = reshape(position, before, after)
        position = np.array([df.values for df in position])
    
    pred = model.predict(position) # Use the model to predict the labels

    num_objects = len(objects)
    # Reshape pred to split into equal parts for each object
    split_size = len(pred) // num_objects
    reshaped_pred = pred.reshape((split_size, num_objects))
    # Create a DataFrame with the reshaped pred and object names as columns
    labels = pd.DataFrame(reshaped_pred, columns=objects)
    
    return labels

In [60]:
def create_autolabels(files, model, objects = ['obj_1', 'obj_2'], bodyparts = ['nose', 'L_ear', 'R_ear', 'head', 'neck', 'body'], rescaling = True, reshaping = False):
    
    for file in files:
        
        # Determine the output file path
        input_dir, input_filename = os.path.split(file)
        parent_dir = os.path.dirname(input_dir)
        
        # Read the file
        position = pd.read_csv(file)
    
        # lets analyze it!
        autolabels = use_model(position, model, objects, bodyparts, rescaling, reshaping)
        
        # Set column names and add a new column "Frame" with row numbers
        autolabels.insert(0, "Frame", autolabels.index + 1)
    
        # Create a filename for the output CSV file
        output_filename = input_filename.replace('_position.csv', '_autolabels.csv')
        output_folder = os.path.join(parent_dir + '/autolabels')
        os.makedirs(output_folder, exist_ok = True)
        output_path = os.path.join(output_folder, output_filename)
        
        # Save autolabels to a CSV file
        autolabels.to_csv(output_path, index=False)
        print(f"Saved autolabels to {output_filename}")

In [61]:
create_autolabels(all_position, loaded_model, objects, bodyparts, rescaling = rescale, reshaping = reshape) # Lets analyze!

469/469 [==============================] - 2s 4ms/step
Saved autolabels to 2023-05_TeNOR_TR1_C1_A_L_autolabels.csv
457/457 [==============================] - 2s 5ms/step
Saved autolabels to 2023-05_TeNOR_TR1_C1_A_R_autolabels.csv
465/465 [==============================] - 2s 5ms/step
Saved autolabels to 2023-05_TeNOR_TR1_C1_B_L_autolabels.csv
464/464 [==============================] - 2s 5ms/step
Saved autolabels to 2023-05_TeNOR_TR1_C1_B_R_autolabels.csv
469/469 [==============================] - 3s 6ms/step
Saved autolabels to 2023-05_TeNOR_TR1_C2_A_L_autolabels.csv
465/465 [==============================] - 2s 5ms/step
Saved autolabels to 2023-05_TeNOR_TR1_C2_A_R_autolabels.csv
460/460 [==============================] - 2s 5ms/step
Saved autolabels to 2023-05_TeNOR_TR1_C2_B_L_autolabels.csv
460/460 [==============================] - 2s 5ms/step
Saved autolabels to 2023-05_TeNOR_TR1_C2_B_R_autolabels.csv
469/469 [==============================] - 3s 5ms/step
Saved autolabels to 2023-